In [1]:
import numpy as np, sys
np.random.seed(1)

from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000])

one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l ] = 1

def tanh(x):
    return np.tanh(x)
    
def tanh2deriv(output):
    return 1 - (output ** 2)
    
def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims = True)
    
    

In [2]:
alpha, iterations = (2,300)
pixels_per_image, num_labels = (784,10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) *
                (input_cols - kernel_cols)) * num_kernels
kernels = 0.02*np.random.random((kernel_rows*kernel_cols,num_kernels)) -0.01
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1

def get_image_section(layer,row_from,row_to,col_from,col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)
    

In [8]:
for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i* batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape
        
        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(   layer_0,
                                            row_start,
                                            row_start+kernel_rows,
                                            col_start,
                                            col_start+kernel_cols)
                sects.append(sect)
                
        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)
        
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2  = softmax(np.dot(layer_1,weights_1_2))
        
        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1])==
                            np.argmax(labelset))
            correct_cnt += _inc
            
        layer_2_delta = (labels[batch_start:batch_end]-layer_2)\
                            / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * \
                            tanh2deriv(layer_1)
        layer_1_delta = dropout_mask 
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
        
    test_correct_cnt = 0
    
    for i in range(len(test_images)):
        
        layer_0 = test_images[i:i+1]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape
        
        sects = list()
        for row_start in range(layer_0.shape[i] - kernel_rows):
            for col_start in range(layer_0.shpae[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                            row_start,
                                            row_start+kernel_rows,
                                            col_start,
                                            col_start+kernel_cols)
                sects.append(sects)
                
        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)
        
        kernel_output = flattened_input.dot(kernels)
        layer_1 =tanh(kernel_output,reshape(es[0],-1))
        layer_2 = np.dot(layer_1,weights_1_2)
        
        test_correct_cnt += int(np.argmax(layer_2)) == np.argmax(test_labels[i:i+1])
                                
    if(j%1 == 0):
        sys.stdout.write("\n"+ \
            "I:" + str(j) + \
            " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+\
            " Train-Acc:"+str(correct_cnt/float(len(images))))

ValueError: shapes (100,10000) and (10816,10) not aligned: 10000 (dim 1) != 10816 (dim 0)

In [9]:
#import mnist
from keras.datasets import mnist
import numpy as np, sys, time

np.random.seed(1) # фиксируем псевдослучайный генератор случайных чисел

x_train = mnist.train_images()[:1000,:,:]
y_train = mnist.train_labels()[:1000,...]

x_test = mnist.test_images()[:2000,:,:]
y_test = mnist.test_labels()[:2000,...]

AttributeError: module 'keras.datasets.mnist' has no attribute 'train_images'

In [ ]:
images, labels = x_train[:1000].reshape(1000, 28*28)/255 , y_train[:1000] # загружаем 1000 изображений и меток (train)
one_hot_labels = np.zeros((1000, 10)) # готовим нулевую матрицу. По строкам обучающие примеры, по столбцам метка
for n,l in enumerate(labels):
    one_hot_labels[n,l] = 1 # для каждой строки в одном из столбцов ставим 1, соответствующую метке (train)
labels = one_hot_labels # переопределяем labels (1000, 10)

test_images, test_labels = x_test.reshape(-1, 28*28)/255, y_test # загружаем 1000 изображений и меток (test)
one_hot_labels_test = np.zeros((x_test.shape[0], 10)) # готовим нулевую матрицу. По строкам обучающие примеры, по столбцам метка
for n, l in enumerate(test_labels): # для каждой строки в одном из столбцов ставим 1, соответствующую метке (test)
    one_hot_labels_test[n,l] = 1
test_labels = one_hot_labels_test

tanh = lambda x: np.tanh(x)
tanh2deriv = lambda output: 1 - (output ** 2)
softmax = lambda x: np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

# устанавливаем параметры
alpha, iterations, pixels_per_image, num_labels = (2, 301, 784, 10)
batch_size = 100

input_rows, input_cols = (28, 28)
kernel_rows, kernel_cols = (3, 3)
num_kernels = 16

# в скрытом слое будут приниматься результаты 26*26 раз просмотров каждого изображения 16-ю ядрами
# то есть по каждому изображению в общей сложности будет 10816 прогнозов 
hidden_size = ((input_rows - kernel_rows+1) * (input_cols - kernel_cols+1)) * num_kernels # (28-3+1)*(28-3+1)*16 = 10816
kernels = 0.02 * np.random.random((kernel_rows*kernel_cols, num_kernels)) - 0.01 # 16 ядер со случайными 9-ю весами (9,16)
# причем веса в диапазоне от -0,01 до 0,01

# инициализируем случайные веса: для каждого из 10816 прогнозов с нейронов скрытого слоя готовим по 10 весов 
weights_1_2 = 0.2*np.random.random((hidden_size, num_labels)) - 0.1 # получаем (10816, 10) 
# причем диапазон весов от -0,1 до 0,1

def get_image_section(layer, row_from, row_to, col_from, col_to):
    '''Функция выбирает подобласть в изображении'''
    sub_section = layer[:, row_from:row_to, col_from:col_to]
    return sub_section.reshape(-1, 1, row_to - row_from, col_to - col_from)

glob_start_time = time.process_time()
for j in range(iterations): # итерируемся по количеству итераций
    start_time = time.process_time()
    correct_cnt = 0 # количество TP-прогнозов
    for i in range(round(images.shape[0]/batch_size)): # бежим по батчам из обучающих примеров
        batch_start, batch_end = ((i * batch_size),((i+1) * batch_size)) # 0-99, 100-199 ...
        layer_0 = images[batch_start:batch_end,:].reshape(-1,28, 28) # подаем на входной слой изображение (100,28,28) 
        # или (128,28,28) в зависимости от размера батча
        
        sects = [] # пустой список для отсмотренных плиточек 3*3
        for row_start in range(layer_0.shape[1] - kernel_rows + 1): # бежим по строкам от 0 до 28-3=25 включительно
                                                            # и смотрим интересующую область сразу по всем изображениям в батче 
            for col_start in range(layer_0.shape[2] - kernel_cols + 1): # бежим по столбцам от 0 до 28-3 включительно
                sect = get_image_section(layer_0, # вырезаем из одного и того же места всех изображений в батче квадрат
                                        row_start, # будет перебираться от 0 до 25 включительно
                                        row_start + kernel_rows, # будет перебираться от 3 до 28 включительно
                                        col_start, # будет перебираться от 0 до 25 включительно
                                        col_start + kernel_cols) # будет перебираться от 3 до 28 включительно
                                        #         print(f'row_start = {row_start} col_start = {col_start}')
                sects.append(sect) # всего д.б. 26*26 = 676 массивов-стопочек размера (128, 1, 3, 3)
        expanded_input = np.concatenate(sects, axis=1) # соединяем массивы горизонтально (присоединяем справа) 
        # получаем (128, 676, 3, 3) - "стену" высотой 128, толщиной 3 и протяженностью 676
        es = expanded_input.shape # (128, 676, 3, 3) просто форма
        flattened_input = expanded_input.reshape(es[0]*es[1], -1) # каждый "плитку" 3*3 вытягиваем в строку, 
        # выкладываем в штабелем и получаем (86528, 9)
        kernel_output = flattened_input.dot(kernels)  # получаем по 16 прогнозов для каждой "плитки"
                                                        # (86528, 9).dot(9,16) = (86528,16)
        layer_1 = tanh(kernel_output.reshape(es[0], -1)) # подаем полученные прогнозы на скрытый слой и сразу для
                                        # последующих умножений меняем форму на (128,10816) и применяем функцию активации,
                                        # которая загоняет полученные входные значения в диапазон от -1 до 1
        dropout_mask = np.random.randint(2, size=layer_1.shape) # создаем маску вида array([[0, 1, 0, ... 0, 0, 1]]),
                                                                # где вероятность единиц и нулей по 50% (128,10816)
        layer_1 *= dropout_mask * 2 # применяем маску, но умноженную на 2 
                        # (т.к. сигнал от слоя 1 примерно в 2 раза уменьшается после применения маски, поэтому его нужно усилить)
                        # (128,10816)
        layer_2 = softmax(layer_1.dot(weights_1_2).reshape(-1,num_labels)) # скалярное перемножение layer_1 (128,10816) 
                                            # на веса (10816,10), получаем выходной слой (128,10) с вероятностями каждого лейбла для
                                            # каждого изображения из батча.
        # теперь посчитаем отклонение
        for k in range(batch_size): # итерируемся по примерам в батче, берем каждое изображение по очереди
            labelset = labels[batch_start:batch_end,:] # правильные ответы
            _inc = int(np.argmax(layer_2[k,:]) == np.argmax(labelset[k,:])) # считаем количество True-Positive ответов
            correct_cnt += _inc # учитываем True-Positive ответ (если есть)
                                                    # в общем показателей correct_cnt для j-той итерации (по j-тому батчу)
        layer_2_delta = (layer_2 - labels[batch_start:batch_end,:])/\
                                            (batch_size * layer_2.shape[0]) # считаем отклонение по выходному слою (128,10)
                                            # делим его на на все количество примеров в батче и количество выходных нейронов layer_2
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * tanh2deriv(layer_1) # считаем отклонение по среднему слою:
                                                        # скалярно перемножаем отклонение по выходному слою (128,10)
                                                        # на транспонированную матрицу весов weights_1_2 (10,10816)
                                                        # получаем вектор (128,10816) и не забываем передать градиент
        layer_1_delta *= dropout_mask # при обратном распространении корректировать обнуленные маской нейроны не следует
                                        # поэтому той же маской обнуляем полученные дельты
        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta) # корректируем веса для выходного слоя: 
                                                            # из матрицы текущих весов (10816,10) вычитаем
                                                            # помноженный на скорость обучения слой 1 транспонированный (10816,128)
                                                            # скалярно помноженный на отклонение по слою 2 (128,10)
                                                            # получаем (10816,10) - такой же формы, как и были наши веса weights_1_2
        # корректируем веса в ядрах
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape) # меняем форму layer_1_delta на форму входного слоя (86528,16) 
        k_update = flattened_input.T.dot(l1d_reshape) # считаем дельту
        kernels -= alpha * k_update # и вычитаем дельту, получаем обновленные веса
        
    test_correct_cnt = 0 # correct_cnt - количество True-Positive ответов

    for i in range(test_images.shape[0]): # бежим по тестовым примерам, тут уже без батчей и корректировки весов разумеется
        layer_0 = test_images[i,:].reshape(1,28,28) # подаем на входной слой изображение (1,28,28)
        sects = []
        for row_start in range(layer_0.shape[1] - kernel_rows + 1): # бежим по строкам от 0 до 28-3 включительно
            for col_start in range(layer_0.shape[2] - kernel_cols + 1): # бежим по столбцам от 0 до 28-3 включительно
                sect = get_image_section(layer_0,
                                row_start, # будет перебираться от 0 до 25 включительно
                                row_start + kernel_rows, # будет перебираться от 3 до 28 включительно
                                col_start, # будет перебираться от 0 до 25 включительно
                                col_start + kernel_cols) # будет перебираться от 3 до 28 включительно
                                #         print(f'row_start = {row_start} col_start = {col_start}')
                sects.append(sect) # всего д.б. 26*26 = 676 массивов размера (128, 1, 3, 3)

        expanded_input = np.concatenate(sects, axis=1) # соединяем массивы горизонтально (присоединяем справа) получаем (128, 676, 3, 3)
        es = expanded_input.shape # (128, 676, 3, 3)
        flattened_input = expanded_input.reshape(es[0]*es[1], -1)
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1)) # 
        layer_2 = layer_1.dot(weights_1_2).reshape(-1,num_labels) # 
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(test_labels[i,:])) # учитываем True-Positive ответ (если есть)
                                                                            # в общем показателей correct_cnt для j-той итерации
    if j % 1 == 0:
        print("Iter:" + str(j) +\
             " Test_Accuracy: " + str(test_correct_cnt/float(test_images.shape[0])) +\
              " Train_Accuracy: " + str(correct_cnt/float(images.shape[0])) + \
              " time: " + str(time.process_time() - start_time) + " sec")
print("Total time: ", time.process_time() - glob_start_time, " sec")

Iter:0 Test_Accuracy: 0.2199 Train_Accuracy: 0.151 time: 23.5625 sec
Iter:1 Test_Accuracy: 0.2564 Train_Accuracy: 0.185 time: 24.546875 sec
Iter:2 Test_Accuracy: 0.3113 Train_Accuracy: 0.239 time: 24.96875 sec
Iter:3 Test_Accuracy: 0.3508 Train_Accuracy: 0.281 time: 25.171875 sec
Iter:4 Test_Accuracy: 0.4056 Train_Accuracy: 0.293 time: 25.203125 sec
Iter:5 Test_Accuracy: 0.4598 Train_Accuracy: 0.352 time: 25.125 sec
Iter:6 Test_Accuracy: 0.5096 Train_Accuracy: 0.395 time: 25.578125 sec
Iter:7 Test_Accuracy: 0.5626 Train_Accuracy: 0.442 time: 25.5 sec
Iter:8 Test_Accuracy: 0.6095 Train_Accuracy: 0.516 time: 24.9375 sec
Iter:9 Test_Accuracy: 0.6523 Train_Accuracy: 0.548 time: 24.765625 sec
Iter:10 Test_Accuracy: 0.6858 Train_Accuracy: 0.59 time: 25.578125 sec
Iter:11 Test_Accuracy: 0.7101 Train_Accuracy: 0.605 time: 26.359375 sec
Iter:12 Test_Accuracy: 0.7302 Train_Accuracy: 0.658 time: 24.6875 sec
Iter:13 Test_Accuracy: 0.7455 Train_Accuracy: 0.671 time: 25.1875 sec
Iter:14 Test_Accurac